<h1> <b>Import libraries </b> </h1>

In [ ]:
import os
import math
import torch
import numpy as np
import seaborn as sn
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.models as models
from torch import nn
from PIL import Image
from tqdm import tqdm
from glob import glob
from gc import collect
from pandas import read_csv
from scipy.io import loadmat
from torch.cuda import empty_cache
from torch.optim import SGD, RMSprop
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, auc
)
from torch.nn import (
    Module, ReLU, Linear, Conv2d, Softmax,
    CrossEntropyLoss, Sequential, Sigmoid, BatchNorm2d,
    MaxPool2d, AdaptiveAvgPool2d, MSELoss
)

<h1> <b>Paths </b> </h1>

In [ ]:
IMG_PATH = "../input/hands-and-palm-images-dataset/Hands/Hands/" #whole images
LABEL_PATH = "../input/hands-and-palm-images-dataset/HandInfo.csv" #information of images
OUT_PATH =  "../input/mat-files-of-hand-images/" #low-frequency and high-frequency images
EYTH_IMG_PATH = "../input/eythdataset/eyth_dataset/images/" #images for segmentation
EYTH_MASK_PATH = "../input/eythdataset/eyth_dataset/masks/" #masks of images for segmentation

In [ ]:
GREEN = "\033[1;32;47m"
RED = "\033[1;31;47m"

<h1> <b>Means and Standard deviations </b> </h1>

In [ ]:
meanImg1, meanImg2, meanImg3 = 240.42/255, 229.6/255, 223.2/255 #mean of pixels of images
meanLow1, meanLow2, meanLow3 = 240.45/255, 229.64/255, 223.24/255 #mean of pixels of low-frequency images
meanHigh = 245.56/255 #mean of pixels of high-frequency images
meanSeg1, meanSeg2, meanSeg3 = 98.17, 91.66, 90.51 #mean of pixels of images for segmentation

In [ ]:
stdImg1, stdImg2, stdImg3 = 32.82/255, 48.86/255, 59.31/255 #std of pixels of images
stdLow1, stdLow2, stdLow3 = 32.38/255, 48.41/255, 58.85/255 #std of pixels of low-frequency images
stdHigh = 28.11/255 #std of pixels of high-frequency images 
stdSeg1, stdSeg2, stdSeg3 = 79.69, 77.25, 77.07 #std of pixels of images for segmentation

<h1> <b>Use GPU </b> </h1>

In [ ]:
device = torch.device("cuda:0")

<h1> <b> Function for extract low or high-frequency image </b> </h1>

In [ ]:
def getPartOfMat(mat,status):
    if status == 'low':
        return mat['O'][:,:,:3]
    elif status == 'high':
        return mat['O'][:,:,3]
    elif status == "normal":
        return mat['O']

<h1> <b> Get images </b> </h1>

In [ ]:
images = glob(IMG_PATH + "*.jpg")
images.sort()

<h1> <b> Get images for Segmentation </b> </h1>

In [ ]:
segImages = glob(EYTH_IMG_PATH + '*.jpg')
segImages.sort()

<h1> <b> Get mask images for Segmentation </b> </h1>

In [ ]:
maskImages = glob(EYTH_MASK_PATH + "*")
maskImages.sort()

<h1> <b> Extracting information and labels of images </b> </h1>

In [ ]:
details = read_csv(LABEL_PATH)
details

<h1> <b> Show stats information of images </b> </h1>

In [ ]:
for col in details.columns:
    if col == "id" or col == "imageName" or col == "age": continue
    types = {}
    for row in details[col]:
        if row in types:
            types[row] += 1
            continue
        else:
            types[row] = 0
                
    plt.title("Information of " + col.capitalize())
    
    withoutXLabels = ["accessories", "nailPolish", "irregularities"]

    if col in withoutXLabels:
        types["Without " + col.capitalize()] = types.pop(0)
        types["With "+ col.capitalize()] = types.pop(1)
        
    else:
        plt.xlabel(col.capitalize())

    plt.ylabel('Number')
        
    plt.bar(range(len(types)), list(types.values()), tick_label=list(types.keys()), color = 'gray')

    plt.ylim(ymin=types[min(types, key=types.get)] * 0.5)
    
    plt.show()

<h1> <b>Get low and high-frequency images</b> </h1>

In [ ]:
outs = glob(OUT_PATH + '*.mat')
outs.sort()

<h1> <b> Delete redundant Cache</b> </h1>

In [ ]:
def deleteRedundantCache(net):
    collect()
    for p in net.parameters():
        if p.grad is not None:
            del p.grad
    empty_cache()

<h1> <b> Function of showing Accuracy and Lost plot</b> </h1>

In [ ]:
import math
print(math.ceil(1.312 * 100) / 100)

In [ ]:
def showPlot(title, trainAccuracy, testAccuracy, trainLoss, testLoss, tpr = None, fpr = None):
    
    minimumValue = min(min(trainAccuracy), min(testAccuracy))
    maximumValue = max(max(trainAccuracy), max(testAccuracy))
    numberOfDivisions = (maximumValue - minimumValue) / 3
    listOf_Yticks = np.arange(minimumValue, math.ceil(maximumValue * 100) / 100, numberOfDivisions)
    
    plt.plot(trainAccuracy,'-o')
    plt.plot(testAccuracy,'-o')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train','Test'])
#     plt.title('Accuracy of ' + title + ' network')
    plt.yticks(listOf_Yticks)
    plt.savefig(title + ' accuracy.png')
    plt.show()
    
    minimumValue = min(min(trainLoss), min(testLoss))
    maximumValue = max(max(trainLoss), max(testLoss))
    numberOfDivisions = (maximumValue - minimumValue) / 3
    listOf_Yticks = np.arange(minimumValue, math.ceil(maximumValue * 100) / 100, numberOfDivisions)

    plt.plot(trainLoss,'-o')
    plt.plot(testLoss,'-o')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train','Test'])
#     plt.title('Loss of ' + title + ' network')
    plt.yticks(listOf_Yticks)
    plt.savefig(title + ' loss.png')
    plt.show()

    if title != 'Segmentation':
        area = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label = "ROC curve (area = %0.5f)" % area)
        plt.plot([0,1], [0,1], 'r--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.legend(loc=4)
        plt.title('ROC of ' + title + ' network')
        plt.savefig(title + ' ROC.eps')
        plt.show()

<h1> <b> Function of showing Confusion Matrix</b> </h1>

In [ ]:
def showConfusionMatrix(title, labels, predicts):
    confusion = confusion_matrix(labels, predicts, labels = [0, 1])
    report = classification_report(labels, predicts, target_names = ["Men", "Women"])
    confusion = np.round(confusion / confusion.astype(np.float).sum(axis=0), 2)

    fig = plt.figure()
    sn.heatmap(confusion, annot=True)
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    fig.suptitle("Confusion Matrix of " + title + " network", fontsize = 12)
    plt.show()
    
    print("\n\n\n")
    
    print(report)

<h1> <b>Dataset for low and high-frequency images </b> </h1>

In [ ]:
class LowHighDataset(Dataset):
    def __init__(self, lowTransform = None, highTransform = None):
        self.df = read_csv(LABEL_PATH)
        self.outs_folder = OUT_PATH
        self.lowTransform = lowTransform
        self.highTransform = highTransform
        self.classToIndex = {"male":0, "female":1}

    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, index):
        filename = self.df["imageName"][index]
        label = self.classToIndex[self.df["gender"][index]]
        
        lowImg = getPartOfMat(loadmat(self.outs_folder + "/" + filename[:-3] + "mat"), "low")
        highImg = getPartOfMat(loadmat(self.outs_folder + "/" + filename[:-3] + "mat"), "high")
        
        if self.lowTransform is not None:
            lowImg = self.lowTransform(lowImg)
            
        if self.highTransform is not None:
            highImg = self.highTransform(highImg)
            
        return lowImg, highImg, label

<h1> <b> Transform low-frequency and high-frequency and main images </b> </h1>

In [ ]:
mainTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((112, 112)),
    transforms.Normalize((meanImg1, meanImg2, meanImg3), (stdImg1, stdImg2, stdImg3))
])

lowTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((meanLow1, meanLow2, meanLow3), (stdLow1, stdLow2, stdLow3))
])

highTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((meanHigh), (stdHigh))
])

<h1> <b> Creating train and test loader for low and high-frequency images </b> </h1>

In [ ]:
dataset = LowHighDataset(lowTransform, highTransform)

batch_size = 130
validation_split = 0.7
shuffle_dataset = True
dataset_size = len(images)
split = int(np.floor(validation_split * dataset_size))
indices = list(range(dataset_size))

if shuffle_dataset: np.random.shuffle(indices)

train_indices, test_indices = indices[:split], indices[split:]
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainLoader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
testLoader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

<h1> <b>Dataset for Segmentation </b> </h1>

In [ ]:
class EYTHDataset(Dataset):
    def __init__(self, image_seg_transform = None, mask_transform = None):
        super(EYTHDataset, self).__init__()
        self.image_path = EYTH_IMG_PATH
        self.mask_path = EYTH_MASK_PATH
        self.image_seg_transform = image_seg_transform
        self.mask_transform = mask_transform
        self.segImages = sorted(os.listdir(self.image_path))
        self.maskImages = sorted(os.listdir(self.mask_path))

    def __len__(self):
        return len(os.listdir(self.image_path))

    def __getitem__(self, idx):

        segImage = Image.open(self.image_path + self.segImages[idx])
        maskImage = Image.open(self.mask_path + self.maskImages[idx])
        if self.image_seg_transform is not None:
            segImage = self.image_seg_transform(segImage)

        if self.mask_transform is not None:
            maskImage = self.mask_transform(maskImage)
            
        return segImage, maskImage

<h1> <b> Transform images and masks for Segmentation </b> </h1>

In [ ]:
image_seg_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize((meanSeg1, meanSeg2, meanSeg3), (stdSeg1, stdSeg2, stdSeg3))
])
mask_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    lambda m: torch.where(m > 0, torch.ones_like(m), torch.zeros_like(m))
])

<h1> <b> Creating train and test loader for images and masks for Segmentation </b> </h1>

In [ ]:
datasetSeg = EYTHDataset(image_seg_transform, mask_transform)

batch_size = 24
validation_split = 0.8
shuffle_dataset = True
dataset_size = len(datasetSeg)
split = int(np.floor(validation_split * dataset_size))
indices = list(range(dataset_size))

if shuffle_dataset: np.random.shuffle(indices)

train_indices, test_indices = indices[:split], indices[split:]
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainLoaderSeg = DataLoader(datasetSeg, batch_size=batch_size, sampler=train_sampler)
testLoaderSeg = DataLoader(datasetSeg, batch_size=batch_size, sampler=test_sampler)

<h1> <b>Dataset for Three-stream </b> </h1>

In [ ]:
class trippleStreamDataset(Dataset):
    def __init__(self, mainTransform = None, lowTransform = None, highTransform = None):
        self.df = read_csv(LABEL_PATH)
        self.outs_folder = OUT_PATH
        self.mainTransform = mainTransform
        self.lowTransform = lowTransform
        self.highTransform = highTransform
        self.classToIndex = {"male":0, "female":1}
        self.imageLst = os.listdir(IMG_PATH)

    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, index):
        filename = self.df["imageName"][index]
        label = self.classToIndex[self.df["gender"][index]]
        side = self.df["aspectOfHand"][index].split()[0]
        
        mainImage = Image.open(IMG_PATH + self.imageLst[index])
        lowImage = getPartOfMat(loadmat(self.outs_folder + "/" + filename[:-3] + "mat"), "low")
        highImage = getPartOfMat(loadmat(self.outs_folder + "/" + filename[:-3] + "mat"), "high")
        
        if self.mainTransform is not None:
            mainImage = self.mainTransform(mainImage)
           
        if self.lowTransform is not None:
            lowImage = self.lowTransform(lowImage)
            
        if self.highTransform is not None:
            highImage = self.highTransform(highImage)
            
        return mainImage, lowImage, highImage, label, side

<h1> <b> Creating train and test loader for Three-stream </b> </h1>

In [ ]:
streamDataset = trippleStreamDataset(mainTransform, lowTransform, highTransform)

batch_size = 60
validation_split = 0.7
shuffle_dataset = True
dataset_size = len(streamDataset)
split = int(np.floor(validation_split * dataset_size))
indices = list(range(dataset_size))

if shuffle_dataset: np.random.shuffle(indices)

train_indices, test_indices = indices[:split], indices[split:]
train_sampler, test_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(test_indices)

trainLoaderStream = DataLoader(streamDataset, batch_size=batch_size, sampler=train_sampler)
testLoaderStream = DataLoader(streamDataset, batch_size=batch_size, sampler=test_sampler)

<h1> <b> Define UNet </b> </h1>

In [ ]:
class DoubleConv(Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()


        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
class UNet(Module):
    def __init__(self, channels, classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = channels
        self.n_classes = classes
        self.bilinear = bilinear

        self.inc = DoubleConv(channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)

        return logits

<h1> <b> Define each block of our version of ResNet network </b> </h1>

In [ ]:
class BasicBlock(Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = BatchNorm2d(planes)
        self.relu = ReLU(inplace=True)
        self.conv2 = Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

<h1> <b> Define our version of ResNet network </b> </h1>

In [ ]:
class ResNet(Module):
    def __init__(self, block, layers, status = "low", num_classes=2):
        super().__init__()
        
        self.attention = None
        
        self.inplanes = 64
        
        inputLayer = 3 if status == "low" else 1 if status == "high" else None

        self.conv1 = Conv2d(inputLayer, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = BatchNorm2d(self.inplanes)
        self.relu = ReLU(inplace=True)
        self.maxpool = MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1])
        self.layer3 = self._make_layer(block, 256, layers[2])
        self.layer4 = self._make_layer(block, 512, layers[3])
        
        self.avgpool = AdaptiveAvgPool2d((1, 1))
        self.fc = Sequential(
            Linear(
                in_features=512,
                out_features=num_classes
            ),
            Sigmoid()
        )

    def _make_layer(self, block, planes, blocks, stride=2):
        downsample = None  
   
        if stride != 1 or self.inplanes != planes:
            downsample = Sequential(
                Conv2d(self.inplanes, planes, 1, stride, bias=False),
                BatchNorm2d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return Sequential(*layers)
    
    def setAttention(self, attention):
        self.attention = attention
    
    def forward(self, x):       
        x = self.conv1(x)           # 224x224
        
        if self.attention != None:
            x = torch.mul(x, self.attention)
        
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)         # 112x112

        x = self.layer1(x)          # 56x56
        x = self.layer2(x)          # 28x28
        x = self.layer3(x)          # 14x14
        x = self.layer4(x)          # 7x7
        
        x = self.avgpool(x)         # 1x1

        x = torch.flatten(x, 1)     # remove 1 X 1 grid and make vector of tensor shape 
        x = self.fc(x)
        return x

In [ ]:
def resnet34(status):
    layers=[2, 3, 4, 2]
    model = ResNet(BasicBlock, layers, status)
    return model

<h1> <b> Define Three-stream </b> </h1>

In [ ]:
class trippleStream(Module):
    def __init__(self, lowNet, highNet, unet):
        super(trippleStream, self).__init__()
        self.unet = unet
        self.lowNet = lowNet
        self.highNet = highNet
        self.fc = Linear(4, 2, bias=True)
    
    def forward(self, xseg, xlow, xhigh):
        xseg = self.unet(xseg)
        
        xseg = Softmax(dim=1)(xseg)
        xseg = xseg.unsqueeze(1)
        xseg = xseg[:,:,1,:,:]
        xseg = xseg.repeat(1,64,1,1)
        self.lowNet.setAttention(xseg)
        self.highNet.setAttention(xseg)
        
        xlow = self.lowNet(xlow)
        xhigh = self.highNet(xhigh)
        x = torch.cat((xlow, xhigh), dim=1)
        x = self.fc(F.relu(x))
        return x

<h1> <b> Functions of train and test of segmentation network</b> </h1>

In [ ]:
def trainSegmentation(epoch, epochs, model, trainLoader, trainAccuracy, trainLoss, opt, criterion):
    running_loss = 0
    total = 0
    correct = 0
    intersection, union = 0, 0
    
    for images, masks in tqdm(trainLoader, desc="Train", colour='green'):
        
        if torch.cuda.is_available(): 
            images = images.to(device, dtype=torch.float32)
            masks = masks.to(device, dtype=torch.long)
        else:
            images = images.to(dtype=torch.float32)
            masks = masks.to(dtype=torch.long)
            
        masks = masks.squeeze(1)
        
        opt.zero_grad()

        with torch.cuda.amp.autocast(enabled=False):
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            loss.backward()
            opt.step()
            
            _, predicted = torch.max(outputs.data, 1)
            total += masks.nelement()
            correct += predicted.eq(masks.data).sum().item()
            
            running_loss += loss.item() / len(trainLoader)
            
    accuracy = 100 * correct / total
    trainAccuracy.append(accuracy)
    trainLoss.append(running_loss)
    
    print("\n" + GREEN + "TRAIN: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss))

In [ ]:
def testSegmentation(epoch, epochs, model, testLoader, testAccuracy, testLoss, opt, criterion):
    running_loss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for images, masks in tqdm(testLoader, desc="Train", colour='red'):

            if torch.cuda.is_available(): 
                images = images.to(device, dtype=torch.float32)
                masks = masks.to(device, dtype=torch.long)
            else:
                images = images.to(dtype=torch.float32)
                masks = masks.to(dtype=torch.long)
            
            masks = masks.squeeze(1)

            with torch.cuda.amp.autocast(enabled=False):
                outputs = model(images)
                loss = criterion(outputs, masks)

                _, predicted = torch.max(outputs.data, 1)
                total += masks.nelement()
                correct += predicted.eq(masks.data).sum().item()

                running_loss += loss.item() / len(testLoader)
        
        accuracy = 100 * correct / total
        testAccuracy.append(accuracy)
        testLoss.append(running_loss)
        
        print("\n" + RED + "TEST: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss))

<h1> <b> Functions of train and test of low and high-frequency network</b> </h1>

In [ ]:
def trainLowHigh(epoch, epochs, model, loader, accuracies, losses, opt, criterion, status):
    running_loss = 0
    total, correct = 0, 0
    for lowInputs, highInputs, labels in tqdm(loader, desc="Train", colour='green'):
        
        inputs = lowInputs if status == "low" else highInputs if status == "high" else None
                
        if torch.cuda.is_available(): 
            inputs = inputs.to(device)
            labels = labels.to(device)

        opt.zero_grad()
        
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()

        running_loss += loss.item() / len(loader)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    accuracies.append(accuracy)
    losses.append(running_loss)
        
    print("\n" + GREEN + "TRAIN: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss))

In [ ]:
def testLowHigh(epoch, epochs, model, loader, accuracies, losses, criterion, status):    
    running_loss = 0
    total, correct = 0, 0
    womenPredicts = []
    allPredicts, allLabels = [] ,[]
    
    with torch.no_grad():
        for lowInputs, highInputs, labels in tqdm(loader, desc="Test", colour='red'):

            inputs = lowInputs if status == "low" else highInputs if status == "high" else None
            if torch.cuda.is_available(): 
                inputs = inputs.to(device)
                labels = labels.to(device)

            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item() / len(loader)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
                        
            womenPredicts.extend([row[1] for row in outputs.detach().cpu().numpy()])
            allLabels.extend(labels.detach().cpu().numpy())
            allPredicts.extend(predicted.detach().cpu().numpy())
        
    fpr, tpr, threshold = roc_curve(allLabels, womenPredicts)             

    accuracy = 100 * correct / total
    accuracies.append(accuracy)
    losses.append(running_loss)
    
    print("\n" + RED + "TEST: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss))
    
    return allLabels, allPredicts, tpr, fpr

<h1> <b> Functions of train and test of three-stream network</b> </h1>

In [ ]:
def trainStream(epoch, epochs, model, loader, accuracies, losses, opt, criterion):
    running_loss = 0
    total, correct = 0, 0
    
    for imgInputs, lowInputs, highInputs, labels, sides in tqdm(loader, desc="Train", colour='green'):
        
        if torch.cuda.is_available(): 
            imgInputs = imgInputs.to(device)
            lowInputs = lowInputs.to(device)
            highInputs = highInputs.to(device)
            labels = labels.to(device)
                        
        opt.zero_grad()

        outputs = model(imgInputs, lowInputs, highInputs)

        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()

        running_loss += loss.item() / len(loader)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()  


    accuracy = 100 * correct / total
    
    accuracies.append(accuracy)
    losses.append(running_loss)
            
    print("\n" + GREEN + "TRAIN: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss))

In [ ]:
def testStream(epoch, epochs, model, loader, accuracies, losses, criterion):    
    running_loss = 0
    total, correct = 0, 0
    totalPalmar, totalDorsal = 0, 0
    correctPalmar, correctDorsal = 0, 0
    womenPredicts = []
    allPredicts, allLabels = [] ,[]

    with torch.no_grad():
        for imgInputs, lowInputs, highInputs, labels, sides in tqdm(loader, desc="Test", colour='red'):
            
            if torch.cuda.is_available(): 
                imgInputs = imgInputs.to(device)
                lowInputs = lowInputs.to(device)
                highInputs = highInputs.to(device)
                labels = labels.to(device)

            outputs = model(imgInputs, lowInputs, highInputs)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item() / len(loader)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            for person, side in enumerate(sides):
                if sides[person] == "palmar":
                    totalPalmar += 1
                    if labels[person] == predicted[person]:
                        correctPalmar += 1
                elif sides[person] == "dorsal":
                    totalDorsal += 1
                    if labels[person] == predicted[person]:
                        correctDorsal += 1

            womenPredicts.extend([row[1] for row in outputs.detach().cpu().numpy()])
            allLabels.extend(labels.detach().cpu().numpy())
            allPredicts.extend(predicted.detach().cpu().numpy())
        
    fpr, tpr, threshold = roc_curve(allLabels, womenPredicts)

    accuracy = 100 * correct / total
    palmarAccuracy = 100 * correctPalmar / totalPalmar
    dorsalAccuracy = 100 * correctDorsal / totalDorsal
    
    accuracies.append(accuracy)
    losses.append(running_loss)
            
    print("\n" + RED + "TEST: Epoch {}/{}, Accuracy: {:.3f}, Loss: {:.3f}, Accuracy of dorsal hands: {:.3f}, Accuracy of palmar hands: {:.3f}\n".format(epoch + 1, epochs, accuracy, running_loss, dorsalAccuracy, palmarAccuracy))
    
    return allLabels, allPredicts, tpr, fpr

<h1> <b> Train and Test hand image segmentations with UNet network</b> </h1>

In [ ]:
unet = UNet(channels = 3, classes = 2)
if torch.cuda.is_available():
    unet = unet.to(device)

In [ ]:
epochs = 35
criterion = CrossEntropyLoss()
opt = RMSprop(unet.parameters(), lr=0.01, momentum=0.9, weight_decay = 1e-7)
scheduler = StepLR(opt, step_size=12, gamma=0.6)

trainAccuracy, trainLoss, testAccuracy, testLoss = [], [], [], []

for epoch in range(epochs):
    trainSegmentation(epoch, epochs, unet, trainLoaderSeg, trainAccuracy, trainLoss, opt, criterion)
    testSegmentation(epoch, epochs, unet, testLoaderSeg, testAccuracy, testLoss, opt, criterion)
    scheduler.step()
    deleteRedundantCache(unet)

In [ ]:
showPlot('Segmentation', trainAccuracy, testAccuracy, trainLoss, testLoss)

<h1> <b>Train and Test Resnet for low-frequency images </b> </h1>

In [ ]:
resnetLow = resnet34("low")
if torch.cuda.is_available(): resnetLow = resnetLow.to(device)

In [ ]:
epochs = 15
criterion = CrossEntropyLoss()
opt = SGD(resnetLow.parameters(), lr=0.09, momentum=0.9, weight_decay = 1e-3)
scheduler = StepLR(opt, step_size=5, gamma=0.7)

trainAccuracy, trainLoss, testAccuracy, testLoss  = [], [], [], []

for epoch in range(epochs):
    trainLowHigh(epoch, epochs, resnetLow, trainLoader, trainAccuracy, trainLoss, opt, criterion, "low")
    labels, predicts, tpr, fpr = testLowHigh(epoch, epochs, resnetLow, testLoader, testAccuracy, testLoss, criterion, "low")
    scheduler.step()
    deleteRedundantCache(resnetLow)

In [ ]:
showConfusionMatrix('low-frequency images', labels, predicts)

In [ ]:
showPlot('low-frequency images', trainAccuracy, testAccuracy, trainLoss, testLoss, tpr, fpr)

<h1> <b>Train and Test Resnet for high-frequency images </b> </h1>

In [ ]:
resnetHigh = resnet34("high")
if torch.cuda.is_available(): resnetHigh = resnetHigh.to(device)

In [ ]:
epochs = 15
criterion = CrossEntropyLoss()
opt = SGD(resnetHigh.parameters(), lr=0.08, momentum=0.9, weight_decay = 1e-3)
scheduler = StepLR(opt, step_size=5, gamma=0.7)

trainAccuracy, trainLoss, testAccuracy, testLoss = [], [], [], []

for epoch in range(epochs):
    trainLowHigh(epoch, epochs, resnetHigh, trainLoader, trainAccuracy, trainLoss, opt, criterion, "high")
    labels, predicts, tpr, fpr = testLowHigh(epoch, epochs, resnetHigh, testLoader, testAccuracy, testLoss, criterion, "high")
    scheduler.step()
    deleteRedundantCache(resnetHigh)

In [ ]:
showConfusionMatrix('high-frequency images', labels, predicts)

In [ ]:
showPlot('high-frequency images', trainAccuracy, testAccuracy, trainLoss, testLoss, tpr, fpr)

 <h1> <b> Train and Test for tripple stream</b> </h1>

In [ ]:
unet = UNet(channels = 3, classes = 2)

threeStream = trippleStream(resnetLow, resnetHigh, unet)

if torch.cuda.is_available():
    unet = unet.to(device)
    threeStream = threeStream.to(device)

In [ ]:
epochs = 25
criterion = CrossEntropyLoss()
opt = SGD(threeStream.parameters(), lr=0.01, momentum=0.9, weight_decay = 1e-4)
scheduler = StepLR(opt, step_size=5, gamma=0.8)

trainAccuracy, trainLoss, testAccuracy, testLoss = [], [], [], []

for epoch in range(epochs):
    trainStream(epoch, epochs, threeStream, trainLoaderStream, trainAccuracy, trainLoss, opt, criterion)
    labels, predicts, tpr, fpr = testStream(epoch, epochs, threeStream, testLoaderStream, testAccuracy, testLoss, criterion)
    scheduler.step()
    deleteRedundantCache(threeStream)

In [ ]:
showConfusionMatrix('three-stream', labels, predicts)

In [ ]:
showPlot('three-stream', trainAccuracy, testAccuracy, trainLoss, testLoss, tpr, fpr)